In [ ]:
import time
import os
import argparse
import cProfile
import pstats
import logging
from datetime import datetime
import numpy as np
import astropy.io.fits as pyfits

import corgihowfsc
import eetc
from howfsc.control.cs import ControlStrategy
from howfsc.model.mode import CoronagraphMode
from howfsc.util.loadyaml import loadyaml
from howfsc.precomp import howfsc_precomputation

import corgihowfsc
from corgihowfsc.utils.howfsc_initialization import get_args, load_files
from corgihowfsc.sensing.Estimator_choice import DefaultEstimator
from corgihowfsc.sensing.DefaultProbes import DefaultProbes
from corgihowfsc.utils.contrast_nomalization import EETCNormalization
from corgihowfsc.gitl.nulling_gitl import nulling_gitl
from corgihowfsc.utils.corgisim_gitl_frames import GitlImage

eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\corgihowfsc' #os.path.dirname(os.path.abspath(corgihowfsc.__file__))
defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')
defjacpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\cgi-howfsc'




In [ ]:
current_datetime = datetime.now()
folder_name = 'gitl_simulation_' + current_datetime.strftime("%Y-%m-%d_%H%M%S")
fits_name = 'final_frames.fits'
fileout_path = os.path.join(os.path.dirname(os.path.dirname(corgihowfsc.__file__)), 'data', folder_name, fits_name)

args = get_args(mode='nfov_band1', precomp='precomp_all_once', num_process=0, num_threads=1, fileout=fileout_path,jacpath=defjacpath)

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath
precomp = args.precomp
num_process = args.num_process
num_threads = args.num_threads

modelpath, cfgfile, jacfile, cstratfile, probefiles, hconffile, n2clistfiles = load_files(args, howfscpath)


# cfg
cfg = CoronagraphMode(cfgfile)

# hconffile
hconf = loadyaml(hconffile, custom_exception=TypeError)

# Define control and estimator strategy
cstrat = ControlStrategy(cstratfile)
estimator = DefaultEstimator()

# Initialize default probes class
probes = DefaultProbes('default')

# Define imager and normalization (counts->contrast) strategy
imager = GitlImage(
    cfg=cfg,         # Your CoronagraphMode object
    cstrat=cstrat,   # Your ControlStrategy object
    hconf=hconf,      # Your host config with stellar properties
    backend='cgi-howfsc',
    cor=mode
)
normalization_strategy = EETCNormalization()



In [ ]:
hconffile

In [ ]:

if nbadpacket < 0:
    raise ValueError('Number of bad packets cannot be less than 0.')
if nbadframe < 0:
    raise ValueError('Number of bad frames cannot be less than 0.')

if isprof:
    pr = cProfile.Profile()
    pass


exptime = 10 # FIXME this should be derived from contrast eventually
contrast = 1e-5 # "starting" value to bootstrap getting we0

# dm1_list, dm2
# Get DM lists
dm1_list, dm2_list, dmrel_list, dm10, dm20 = probes.get_dm_probes(cfg, probefiles)
nlam = len(cfg.sl_list)
ndm = 2 * len(dmrel_list) + 1

# cstratfile
# cstrat = ControlStrategy(cstratfile)

# nrow, ncol, croplist
nrow = 153
ncol = 153
lrow = 436
lcol = 436
croplist = [(lrow, lcol, nrow, ncol)]*(nlam*ndm)
subcroplist = [(lrow, lcol, nrow, ncol)]*(nlam)
nrowperpacket = 3 # only used by packet-drop testing

# prev_exptime_list
prev_exptime_list = [exptime]*(nlam*ndm)

# jac, jtwj_map, n2clist
if precomp in ['precomp_all_once']:
    t0 = time.time()
    jac, jtwj_map, n2clist = howfsc_precomputation(
        cfg=cfg,
        dmset_list=[dm10, dm20],
        cstrat=cstrat,
        subcroplist=subcroplist,
        jacmethod='fast',
        do_n2clist=True,
        num_process=num_process,
        num_threads=num_threads,
    )
    t1 = time.time()
    print('Jac/JTWJ/n2clist computation time: ' + str(t1-t0) + ' seconds')

In [ ]:
output = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\data\nfov_band1_jac_full.fits'

In [ ]:
pyfits.writeto(output, jac)